In [1]:
import pandas as pd
import sqlalchemy
import geopandas as gpd
import seaborn as sns
from shapely import wkt
from itertools import cycle, islice

C:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.11.4-CAPI-1.17.4) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
df = pd.read_csv(r'..\..\..\outputs\network\network_results.csv')

# Congested network components by time of day
df.columns

# Get freeflow from 20to5 period

# Exclude trips taken on non-designated facilities (facility_type == 0)
# These are artificial (weave lanes to connect HOV) or for non-auto uses 
df = df[df['data3'] != 0]    # data3 represents facility_type

# calculate total link VMT and VHT
df['VMT'] = df['@tveh']*df['length']
df['VHT'] = df['@tveh']*df['auto_time']/60

# Define facility type
df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

# Calculate delay
# Select links from overnight time of day
delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

# Merge delay field back onto network link df
df = pd.merge(df, delay_df, on='ij', how='left')

# Calcualte hourly delay
df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

# Calcualte speed as a percent of freeflow
df['flow_index'] = df['auto_time']/df['freeflow_time']

df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})

df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,1], labels=['Severe','Heavy','Moderate','Light'])

In [3]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf


In [4]:
def format_df(df, col_list, col_name, index_list, index_name):
    
    # Re-order columns
    df = df[col_list]
    # Re-order index
    df = df.reindex(index_list)
    # Set visible column and index names
    df.columns.name = col_name
    df.index.name = index_name
    
    return df


In [5]:
# set chart style
sns.set_style("dark")
sns.set_context("notebook", font_scale=1.5, 
                rc={"lines.linewidth": 1.5})
colors = ['#91268F','#F05A28','#8CC63E','#00A7A0','#4C4C4C', '#BFE9E7']
sns.set(font="Pragmatica Condensed Light")

In [6]:
# Load regional geographies layer to intersect with network shapefile
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'cities', version, crs=crs, is_table=False)

# Intersect geography this with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
gdf_intersect = gpd.overlay(gdf_network, gdf_shp, how="union", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

### IMPORTANT
# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

C:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df = df[['@tveh','congestion_category','i_node','j_node','tod','total_delay']].merge(gdf_intersect, 
                                                             left_on=['i_node','j_node'],
                                                            right_on=['i','j'],
                                                                  how='left')
df['VMT'] = df['new_length']*df['@tveh']

# Assume speed is constant or assigned link
# When links are split, we take proportionate delay by length

# calculate total delay
df['delay'] = (df['new_length']/df['length'])*df['total_delay']
# Note that we won't get all of the data because some links are outside the region boundaries
# Optionally, scale these numbers by the total delay field total_delay for consistency
df['scaled_delay'] = df['delay']*(df['total_delay'].sum()/df['delay'].sum())

# Congestion by City
### Total VMT & Percent of VMT by Congestion Level

- **Severe: < 25% Speed Limit** (e.g., less than 15 mph on a road with speed limit of 60 mph)
- **Heavy: 25-50% Speed Limit** (e.g., 15-30 mph on a road with speed limit of 60 mph)
- **Moderate: 50-70% Speed Limit** (e.g, 30-42 mph on a road with speed limit of 60 mph)

#### All Day

In [8]:
pd.options.display.max_rows = None

In [9]:
df.rename(columns={'city_name': 'City'}, inplace=True)
df['City'] = df['City'].fillna('Not in City Limits')

# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,28,80,"27,124","153,620"
Arlington,225,"1,631","16,372","295,278"
Auburn,"4,276","60,986","129,161","1,643,976"
Bainbridge Island,230,"1,988","2,013","206,843"
Beaux Arts,0,0,0,450
Bellevue,"19,747","215,341","469,346","3,124,101"
Black Diamond,130,745,"1,387","50,394"
Bonney Lake,"1,043","13,547","19,872","225,068"
Bothell,"21,381","107,391","264,459","1,325,063"


In [10]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,0.0%,15.0%,84.9%
Arlington,0.1%,0.5%,5.2%,94.2%
Auburn,0.2%,3.3%,7.0%,89.4%
Bainbridge Island,0.1%,0.9%,1.0%,98.0%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.5%,5.6%,12.3%,81.6%
Black Diamond,0.2%,1.4%,2.6%,95.7%
Bonney Lake,0.4%,5.2%,7.7%,86.7%
Bothell,1.2%,6.2%,15.4%,77.1%


#### AM Peak (7 to 8 AM)

In [11]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0,3,"4,307","8,217"
Arlington,15,210,"1,254","20,867"
Auburn,420,"9,598","22,893","103,315"
Bainbridge Island,23,142,249,"15,091"
Beaux Arts,0,0,0,32
Bellevue,"1,992","37,877","60,201","188,373"
Black Diamond,21,69,261,"3,484"
Bonney Lake,312,"1,097","3,248","12,533"
Bothell,725,"25,707","28,807","61,618"


In [12]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.0%,0.0%,34.4%,65.6%
Arlington,0.1%,0.9%,5.6%,93.4%
Auburn,0.3%,7.0%,16.8%,75.8%
Bainbridge Island,0.1%,0.9%,1.6%,97.3%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,0.7%,13.1%,20.9%,65.3%
Black Diamond,0.5%,1.8%,6.8%,90.8%
Bonney Lake,1.8%,6.4%,18.9%,72.9%
Bothell,0.6%,22.0%,24.7%,52.7%


#### PM Peak (5 to 6 PM)

In [13]:
pd.options.display.float_format = '{:0,.0f}'.format
df_pm = df[df['tod'] == '17to18']
_df = df_pm.pivot_table(index='congestion_category',columns='City',
               aggfunc='sum',values='VMT')
_df = _df.T
_df.loc['Total',:] = _df.sum() 
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,14,3,"4,447","8,851"
Arlington,33,102,"1,820","23,133"
Auburn,436,"10,554","11,171","128,891"
Bainbridge Island,18,196,157,"17,539"
Beaux Arts,0,0,0,35
Bellevue,"4,602","33,243","60,913","214,227"
Black Diamond,24,54,104,"4,369"
Bonney Lake,144,"1,777","1,711","17,210"
Bothell,"5,387","14,656","39,514","73,417"


In [14]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df.drop('Total', axis=0)
_sum = _df.sum(axis=1)
for col in _df.columns.categories:
    _df[col] = _df[col]/_sum
_df

congestion_category,Severe,Heavy,Moderate,Light
City,,,,
Algona,0.1%,0.0%,33.4%,66.5%
Arlington,0.1%,0.4%,7.3%,92.2%
Auburn,0.3%,7.0%,7.4%,85.3%
Bainbridge Island,0.1%,1.1%,0.9%,97.9%
Beaux Arts,0.0%,0.0%,0.0%,100.0%
Bellevue,1.5%,10.6%,19.5%,68.4%
Black Diamond,0.5%,1.2%,2.3%,96.0%
Bonney Lake,0.7%,8.5%,8.2%,82.6%
Bothell,4.1%,11.0%,29.7%,55.2%


# Network Delay
Totals scaled to match network totals. 

In [15]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['scaled_delay']]
_df.loc['Total',:] = _df.sum() 
_df

,scaled_delay
City,
Algona,611
Arlington,239
Auburn,"4,145"
Bainbridge Island,74
Beaux Arts,0
Bellevue,"15,564"
Black Diamond,28
Bonney Lake,805
Bothell,"8,798"


Totals unscaled. Some links cross outside the region and delay on these links are excluded at boundaries

In [16]:
# Percent of VMT by congestion category
pd.options.display.float_format = '{:0,.0f}'.format
_df = df.groupby('City').sum()[['delay']]
_df.loc['Total',:] = _df.sum() 
_df

,delay
City,
Algona,519
Arlington,203
Auburn,"3,520"
Bainbridge Island,63
Beaux Arts,0
Bellevue,"13,220"
Black Diamond,24
Bonney Lake,684
Bothell,"7,473"
